##Preparation

In [3]:
%pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff

from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

##Question 2

In [5]:
alpha_records = []
for record in SeqIO.parse('assignment1-alpha_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.count_amino_acids()
  d['length'] = prot.length
  alpha_records.append(d)
alpha_df = pd.DataFrame(alpha_records)
print(alpha_df)
beta_records = []
for record in SeqIO.parse('assignment1-beta_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.count_amino_acids()
  d['length'] = prot.length
  beta_records.append(d)
beta_df = pd.DataFrame(beta_records)
print(beta_df)

      A  C   D   E   F   G   H   I   K   L  ...   N   P   Q   R   S   T   V  \
0     5  0   0   1   4   1   1   4   0   2  ...   2   4   3   2   2   5   5   
1     6  0   2   0   5   4   0   5   1  11  ...   1   2   3   1   4   1  11   
2    53  3  10   9  21  40  14  38  10  69  ...  12  15   8  15  23  13  29   
3    48  6  12  16  25  46   4  49  18  53  ...  14  12   6  13  30  20  38   
4    48  2  11  13  31  47  10  29  12  41  ...  16  15   8  12  19  21  31   
..   .. ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
695  21  5  28  19  23  30   9  31  22  51  ...  24  20  11  35  32  32  33   
696  39  1   8  16  21  31   1  28   9  69  ...  11  20  14   9  21  10  32   
697  16  1   2   6  11   7   1  11   2  19  ...   1   5   2   8   6   5  14   
698  63  2  15  17  32  42   9  45  16  78  ...   8  29  16  14  21  29  40   
699  13  1   3   3   5   6   0  10   3  17  ...   7   0   4   4   2   4  11   

      W   Y  length  
0     1   1      44  
1     2

In [6]:
tmh_sum = alpha_df.sum(axis=0)
tmh_comp = tmh_sum.iloc[:-1]/tmh_sum.iloc[-1]
print(tmh_comp)
tmb_sum = beta_df.sum(axis=0)
tmb_comp = tmb_sum.iloc[:-1]/tmb_sum.iloc[-1]
print(tmb_comp)

A    0.083721
C    0.013473
D    0.039381
E    0.047744
F    0.056452
G    0.071620
H    0.022181
I    0.065857
K    0.044086
L    0.118276
M    0.028000
N    0.035601
P    0.046773
Q    0.032777
R    0.045215
S    0.067427
T    0.053207
V    0.074345
W    0.018591
Y    0.034161
dtype: float64
A    0.077347
C    0.005321
D    0.066632
E    0.047446
F    0.042715
G    0.093541
H    0.019934
I    0.040946
K    0.048122
L    0.079648
M    0.016194
N    0.060836
P    0.037695
Q    0.042974
R    0.049992
S    0.077505
T    0.066690
V    0.060592
W    0.016899
Y    0.048971
dtype: float64


##Question 3

In [7]:
X_df = {}
X = {}
alpha_records = []
for record in SeqIO.parse('assignment1-alpha_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.get_amino_acids_percent()
  alpha_records.append(d)
X_df["alpha"] = pd.DataFrame(alpha_records)
X["alpha"] = pd.DataFrame(alpha_records).to_numpy()
beta_records = []
for record in SeqIO.parse('assignment1-beta_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.get_amino_acids_percent()
  beta_records.append(d)
X_df["beta"] = pd.DataFrame(beta_records)
X["beta"] = pd.DataFrame(beta_records).to_numpy()

In [8]:
mu_alphas = {}
mu_betas = {}
var_alphas = {}
var_betas = {}
fdrs = {}

for col in X_df["alpha"].columns:
  mu_a = np.mean(X_df["alpha"][col])
  mu_b = np.mean(X_df["beta"][col])
  var_a = np.var(X_df["alpha"][col])
  var_b = np.var(X_df["beta"][col])
  mu_alphas[col] = mu_a
  mu_betas[col] = mu_b
  var_alphas[col] = var_a
  var_betas[col] = var_b
  fdr = (mu_a - mu_b)**2/(var_a + var_b)
  fdrs[col] = fdr

fdr_s = pd.Series(fdrs)
fdr_s.sort_values(ascending=False)

D    1.422156
N    0.980141
L    0.831670
I    0.792431
M    0.583876
Y    0.290869
G    0.278405
F    0.269152
Q    0.254863
T    0.239427
V    0.236755
C    0.210791
S    0.145753
P    0.141287
A    0.029176
R    0.021038
K    0.018480
E    0.016473
W    0.013881
H    0.000159
dtype: float64

In [9]:
mus = {}
sigmas = {}
for cls, val in X.items():
  mus[cls] = np.mean(val, axis=0)
  sigmas[cls] = np.var(val, axis=0)

fdr = (mus['alpha'] - mus['beta'])**2/(sigmas['alpha'] + sigmas['beta'])
sort = np.argsort(fdr)
sort

array([ 6, 18,  3,  8, 14,  0, 12, 15,  1, 17, 16, 13,  4,  5, 19, 10,  7,
        9, 11,  2])

In [10]:
mus = {}
for cls, val in X.items():
  mus[cls] = np.mean(val, axis=0)

sks = []
for cls, mu in mus.items():
  sub = np.subtract(X[cls], mu)
  sks.append(sub.T.dot(sub))
sks = np.array(sks)
sw = np.sum(sks, axis=0)

Nk = {}
count = 0
for cls, val in X.items():
  Nk[cls] = val.shape[0]
  count += np.sum(val, axis=0)

N = sum(list(Nk.values()))
m = count / N

sb = []
for cls, mu in mus.items():
  sub = mu - m
  sb.append(np.multiply(Nk[cls], np.outer(sub, sub.T)))
sb = np.sum(sb, axis=0)

mat = np.linalg.pinv(sw).dot(sb)
eigval, eigvec = np.linalg.eig(mat)

eiglist = [(eigval[i], eigvec[:, i]) for i in range(len(eigval))]
eiglist = sorted(eiglist, key=lambda x: x[0], reverse=True)

w = np.array([eiglist[0][1]])
w = np.asarray(w).T
sort = np.argsort(np.absolute(w.T[0]))
print(sort)

[14 17  4  0 18  3  6  9  8 12 10 15  7  5 16 19 13  1  2 11]


## Question 4

In [11]:
aa = 'ACDEFGHIKLMNPQRSTVWY'
dpts = [aa1 + aa2 for aa1 in aa for aa2 in aa]

alpha_dpts = []
for record in SeqIO.parse('assignment1-alpha_seq-1.fasta', 'fasta'):
  counts = {}
  for dpt in dpts:
    counts[dpt] = record.seq.count_overlap(dpt)
  alpha_dpts.append(counts)
alpha_dpts_df = pd.DataFrame(alpha_dpts)
tmh_dpt_sum = alpha_dpts_df.sum(axis=0)
print(tmh_dpt_sum)
dpt_tmp = {}
dpt_ser = {}
for aa1 in aa:
  col = {}
  for aa2 in aa:
    count = tmh_dpt_sum.loc[aa1 + aa2]
    aa1_c = tmh_sum.loc[aa1]
    aa2_c = tmh_sum.loc[aa2]
    prop = float(count)/float(aa1_c + aa2_c)
    col[aa2] = prop
    dpt_ser[aa1 + aa2] = prop
  dpt_tmp[aa1] = col
tmh_dpt_mat = pd.DataFrame(dpt_tmp)
tmh_dpt_ser = pd.Series(dpt_ser)
print(tmh_dpt_mat)
beta_dpts = []
for record in SeqIO.parse('assignment1-beta_seq-1.fasta', 'fasta'):
  counts = {}
  for dpt in dpts:
    counts[dpt] = record.seq.count_overlap(dpt)
  beta_dpts.append(counts)
beta_dpts_df = pd.DataFrame(beta_dpts)
tmb_dpt_sum = beta_dpts_df.sum(axis=0)
print(tmb_dpt_sum)
dpt_tmp = {}
dpt_ser = {}
for aa1 in aa:
  col = {}
  for aa2 in aa:
    count = tmb_dpt_sum.loc[aa1 + aa2]
    aa1_c = tmb_sum.loc[aa1]
    aa2_c = tmb_sum.loc[aa2]
    prop = float(count)/float(aa1_c + aa2_c)
    col[aa2] = prop
    dpt_ser[aa1 + aa2] = prop
  dpt_tmp[aa1] = col
tmb_dpt_mat = pd.DataFrame(dpt_tmp)
tmb_dpt_ser = pd.Series(dpt_ser)
print(tmb_dpt_mat)

AA    2330
AC     269
AD     729
AE     942
AF    1288
      ... 
YS     623
YT     479
YV     590
YW     182
YY     395
Length: 400, dtype: int64
          A         C         D         E         F         G         H  \
A  0.052753  0.009751  0.023559  0.028433  0.031968  0.039023  0.011885   
C  0.010492  0.014631  0.008966  0.007803  0.012524  0.010024  0.008400   
D  0.022450  0.007603  0.030997  0.030154  0.018949  0.023873  0.013117   
E  0.027164  0.007184  0.029371  0.041012  0.019538  0.027568  0.011765   
F  0.034834  0.012524  0.020412  0.017391  0.034685  0.030578  0.013306   
G  0.039413  0.012252  0.022883  0.026361  0.032975  0.040573  0.014873   
H  0.012851  0.009569  0.014225  0.016265  0.011956  0.014792  0.076654   
I  0.037916  0.011851  0.024099  0.023627  0.035273  0.036703  0.011368   
K  0.022988  0.008826  0.025117  0.035792  0.020437  0.025884  0.013101   
L  0.054089  0.012143  0.023902  0.027995  0.041202  0.046316  0.014791   
M  0.023346  0.006399  0.012

In [12]:
tmh_pref = tmh_dpt_ser.nlargest(5)
print(tmh_pref)
tmb_pref = tmb_dpt_ser.nlargest(5)
print(tmb_pref)

HH    0.076654
LL    0.063175
LA    0.054558
AL    0.054089
AA    0.052753
dtype: float64
HH    0.116522
AG    0.049066
LG    0.048995
GG    0.047509
SL    0.046399
dtype: float64


In [75]:
alpha_rpp_cols = {}
for dpt in alpha_dpts_df:
  alpha_rpp_cols[dpt] = alpha_dpts_df[dpt]/(alpha_df[dpt[0]] + alpha_df[dpt[1]])
alpha_rpp_df = pd.DataFrame(alpha_rpp_cols)
alpha_rpp_df.fillna(0, inplace=True)
beta_rpp_cols = {}
for dpt in beta_dpts_df:
  beta_rpp_cols[dpt] = beta_dpts_df[dpt]/(beta_df[dpt[0]] + beta_df[dpt[1]])
beta_rpp_df = pd.DataFrame(beta_rpp_cols)
beta_rpp_df.fillna(0, inplace=True)

In [76]:
mu_alphas = {}
mu_betas = {}
var_alphas = {}
var_betas = {}
fdrs = {}

for col in alpha_rpp_df.columns:
  mu_a = np.mean(alpha_rpp_df[col])
  mu_b = np.mean(beta_rpp_df[col])
  var_a = np.var(alpha_rpp_df[col])
  var_b = np.var(beta_rpp_df[col])
  mu_alphas[col] = mu_a
  mu_betas[col] = mu_b
  var_alphas[col] = var_a
  var_betas[col] = var_b
  fdr = (mu_a - mu_b)**2/(var_a + var_b)
  fdrs[col] = fdr

fdr_s = pd.Series(fdrs)
fdr_s.sort_values(ascending=False)

LL    4.984498e-01
IL    3.572098e-01
LI    3.464884e-01
VL    3.106950e-01
II    2.938419e-01
          ...     
AP    2.600720e-06
LY    2.390641e-06
KH    2.374180e-06
DM    7.252604e-07
QH    9.044232e-08
Length: 400, dtype: float64

## Question 5

In [14]:
alpha_records = []
for record in SeqIO.parse('assignment1-alpha_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.get_amino_acids_percent()
  alpha_records.append(d)
tmh_comp_df = pd.DataFrame(alpha_records)
print(tmh_comp_df)
beta_records = []
for record in SeqIO.parse('assignment1-beta_seq-1.fasta', 'fasta'):
  prot = ProteinAnalysis(record.seq)
  d = prot.get_amino_acids_percent()
  beta_records.append(d)
tmb_comp_df = pd.DataFrame(beta_records)
print(tmb_comp_df)

            A         C         D         E         F         G         H  \
0    0.113636  0.000000  0.000000  0.022727  0.090909  0.022727  0.022727   
1    0.096774  0.000000  0.032258  0.000000  0.080645  0.064516  0.000000   
2    0.128954  0.007299  0.024331  0.021898  0.051095  0.097324  0.034063   
3    0.107143  0.013393  0.026786  0.035714  0.055804  0.102679  0.008929   
4    0.115663  0.004819  0.026506  0.031325  0.074699  0.113253  0.024096   
..        ...       ...       ...       ...       ...       ...       ...   
695  0.044397  0.010571  0.059197  0.040169  0.048626  0.063425  0.019027   
696  0.104839  0.002688  0.021505  0.043011  0.056452  0.083333  0.002688   
697  0.121212  0.007576  0.015152  0.045455  0.083333  0.053030  0.007576   
698  0.119093  0.003781  0.028355  0.032136  0.060491  0.079395  0.017013   
699  0.128713  0.009901  0.029703  0.029703  0.049505  0.059406  0.000000   

            I         K         L         M         N         P         Q  

In [15]:
tmh_comp_ov = tmh_comp.to_numpy()
tmb_comp_ov = tmb_comp.to_numpy()
tmh_comp_np = tmh_comp_df.to_numpy()

tmh_der_tmh = np.sum(np.abs(tmh_comp_np - tmh_comp_ov), axis=1)
tmh_der_tmb = np.sum(np.abs(tmh_comp_np - tmb_comp_ov), axis=1)
tmh_pred = tmh_der_tmh < tmh_der_tmb
print(tmh_pred)

[ True  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True  True False False  True False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False False  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True False  True  True  True  True
  True False  True  True  True  True False False  True  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True False False False  True False  True False  True False False
  True  True  True  True  True  True False False False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True False  True  True  True  True  True  T

In [16]:
tp = np.count_nonzero(tmh_pred)
fn = len(tmh_pred) - tp
tpr = tp/len(tmh_pred)
fnr = fn/len(tmh_pred)
print(tp, tpr)
print(fn, fnr)

627 0.8957142857142857
73 0.10428571428571429


## Question 6

In [17]:
tmb_comp_np = tmb_comp_df.to_numpy()

tmb_der_tmb = np.sum(np.abs(tmb_comp_np - tmb_comp_ov), axis=1)
tmb_der_tmh = np.sum(np.abs(tmb_comp_np - tmh_comp_ov), axis=1)
tmb_pred = tmb_der_tmb < tmb_der_tmh
print(tmb_pred)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True False  True  True False  True False  True False False  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
 False  True  True  True  True  True  True  True False  True  True  True]


In [18]:
tn = np.count_nonzero(tmb_pred)
fp = len(tmb_pred) - tn
tnr = tn/len(tmb_pred)
fpr = fp/len(tmb_pred)
print(tn, tnr)
print(fp, fpr)

131 0.9097222222222222
13 0.09027777777777778


##Question 7

In [19]:
sensitivity = tpr
specificity = tnr
accuracy = (tp + tn)/(tp + tn + fp + fn)

print(tp, tn, fp, fn)
print(sensitivity, specificity, accuracy)

627 131 13 73
0.8957142857142857 0.9097222222222222 0.8981042654028436


##Question 8

In [20]:
split = 0.5
tmh_tr_df = alpha_df.sample(frac=split, random_state=2220)
tmh_ts_df = alpha_df[~alpha_df.isin(tmh_tr_df)].dropna()
tmb_tr_df = beta_df.sample(frac=split, random_state=2220)
tmb_ts_df = beta_df[~beta_df.isin(tmb_tr_df)].dropna()

In [21]:
tmh_s_sum = tmh_tr_df.sum(axis=0)
tmh_s_comp = tmh_s_sum.iloc[:-1]/tmh_s_sum.iloc[-1]
print(tmh_s_comp)
tmb_s_sum = tmb_tr_df.sum(axis=0)
tmb_s_comp = tmb_s_sum.iloc[:-1]/tmb_s_sum.iloc[-1]
print(tmb_s_comp)

A    0.085959
C    0.013185
D    0.039141
E    0.047852
F    0.055515
G    0.073727
H    0.022184
I    0.065429
K    0.042972
L    0.118227
M    0.028185
N    0.035029
P    0.046848
Q    0.033058
R    0.045231
S    0.066684
T    0.053728
V    0.074384
W    0.018057
Y    0.034113
dtype: float64
A    0.076367
C    0.006480
D    0.064579
E    0.046675
F    0.041703
G    0.095472
H    0.020754
I    0.039999
K    0.049189
L    0.080920
M    0.015921
N    0.060585
P    0.038155
Q    0.043099
R    0.050027
S    0.077316
T    0.066060
V    0.062373
W    0.016284
Y    0.048043
dtype: float64


In [22]:
tmh_s_comp_df = tmh_tr_df.iloc[:, :-1].div(tmh_tr_df.length, axis=0)
tmb_s_comp_df = tmb_tr_df.iloc[:, :-1].div(tmb_tr_df.length, axis=0)

In [23]:
tmh_s_comp_ov = tmh_s_comp.to_numpy()
tmb_s_comp_ov = tmb_s_comp.to_numpy()
tmh_s_comp_np = tmh_s_comp_df.to_numpy()
tmb_s_comp_np = tmb_s_comp_df.to_numpy()

tmh_s_der_tmh = np.sum(np.abs(tmh_s_comp_np - tmh_s_comp_ov), axis=1)
tmh_s_der_tmb = np.sum(np.abs(tmh_s_comp_np - tmb_s_comp_ov), axis=1)
tmh_s_pred = tmh_s_der_tmh < tmh_s_der_tmb
print(tmh_s_pred)
tmb_s_der_tmb = np.sum(np.abs(tmb_s_comp_np - tmb_s_comp_ov), axis=1)
tmb_s_der_tmh = np.sum(np.abs(tmb_s_comp_np - tmh_s_comp_ov), axis=1)
tmb_s_pred = tmb_s_der_tmb < tmb_s_der_tmh
print(tmb_s_pred)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True False  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False  True  True  True False  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True False  True  True False  True  True  True  True
  True  True  True  True  True  True  True False  T

In [24]:
tp = np.count_nonzero(tmh_s_pred)
fn = len(tmh_s_pred) - tp
tpr = tp/len(tmh_s_pred)
fnr = fn/len(tmh_s_pred)
print(tp, tpr)
print(fn, fnr)

tn = np.count_nonzero(tmb_s_pred)
fp = len(tmb_s_pred) - tn
tnr = tn/len(tmb_s_pred)
fpr = fp/len(tmb_s_pred)
print(tn, tnr)
print(fp, fpr)

sensitivity = tpr
specificity = tnr
accuracy = (tp + tn)/(tp + tn + fp + fn)

print(tp, tn, fp, fn)
print(sensitivity, specificity, accuracy)

316 0.9028571428571428
34 0.09714285714285714
62 0.8611111111111112
10 0.1388888888888889
316 62 10 34
0.9028571428571428 0.8611111111111112 0.8957345971563981


##Question 9

In [83]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [87]:
X = pd.concat([alpha_rpp_df, beta_rpp_df], axis=0, ignore_index=True)
X = X.to_numpy()

y = []
for i in range(len(alpha_rpp_df)):
  y.append(0)
for i in range(len(beta_rpp_df)):
  y.append(1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=2220)

clf = SVC()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(f1_score(clf.predict(X_test), y_test, pos_label=0))

0.96415770609319
0.9790794979079498


In [88]:
X0_a = alpha_rpp_df
X0_b = beta_rpp_df
X1_a = tmh_comp_df
X1_b = tmb_comp_df
X_a = pd.concat([X0_a, X1_a], axis=1)
X_b = pd.concat([X0_b, X1_b], axis=1)
X = pd.concat([X_a, X_b], axis=0, ignore_index=True)
X = X.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=2220)

clf = SVC()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(f1_score(clf.predict(X_test), y_test, pos_label=0))

0.967741935483871
0.9812108559498957


In [89]:
rpp = ["LL", "IL", "LI", "VL", "II"]
aa = ["D", "N", "L", "I", "M"]
X0_a = alpha_rpp_df[rpp]
X0_b = beta_rpp_df[rpp]
X1_a = tmh_comp_df[aa]
X1_b = tmb_comp_df[aa]
X_a = pd.concat([X0_a, X1_a], axis=1)
X_b = pd.concat([X0_b, X1_b], axis=1)
X = pd.concat([X_a, X_b], axis=0, ignore_index=True)
X = X.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=2220)

clf = SVC()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(f1_score(clf.predict(X_test), y_test, pos_label=0))

0.9498207885304659
0.9703389830508474
